In [10]:
import os
import geopandas as gpd
import h3
from shapely.geometry import Point, Polygon
import pandas as pd
import pydeck

In [48]:
## 서울시 H3 그리드 불러오기
seoul_grid = gpd.read_file("data/그리드/seoul_grid3.shp")

### 건물

In [42]:
## 서울시 건물 정보 불러오기
build = gpd.read_file("data/F_FAC_BUILDING_서울/F_FAC_BUILDING_11_202210.shp", encoding = 'euc-kr')

In [43]:
## 건물명칭 / 통합높이 / geometry 추출
build_height = build[['BLD_NM', 'HEIGHT', 'geometry']]

## 건물의 중심점 추출하여 Point로 변환
build_height['geometry'] = build_height['geometry'].centroid

c:\Users\user\Anaconda3\envs\geo\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [44]:
## 결측값 채우기 (서울시 외 H3 그리드)
build_height['HEIGHT'] = build_height['HEIGHT'].fillna(0.0)
build_height['BLD_NM'] = build_height['BLD_NM'].fillna("정보없음")

In [45]:
build_height

,BLD_NM,HEIGHT,geometry
0,정보없음,9.10,POINT (201839.038 452776.991)
1,정보없음,0.00,POINT (201343.829 452777.379)
2,정보없음,0.00,POINT (201375.177 452774.201)
3,정보없음,0.00,POINT (201388.462 452771.013)
4,정보없음,0.00,POINT (200468.269 452776.403)
...,...,...,...
697882,정보없음,18.00,POINT (201821.763 451827.109)
697883,DMC래미안클라시스,11.25,POINT (192517.135 452812.706)
697884,정보없음,10.20,POINT (202329.717 459965.755)
697885,정보없음,0.00,POINT (201343.914 458685.038)


In [46]:
## 건물 정보 좌표계 변경
build_height = build_height.to_crs(epsg = 4326)

In [52]:
## 그리드와 건물 정보 병합
build_grid = gpd.sjoin(seoul_grid, build_height, how = "left", op = "intersects")
build_grid.head(3)

c:\Users\user\Anaconda3\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,h3_9,geometry,index_right,BLD_NM,HEIGHT
0,8930e0a5423ffff,"POLYGON ((126.79014 37.60392, 126.78916 37.602...",NaN,NaN,NaN
1,8930e1d8127ffff,"POLYGON ((126.97254 37.56289, 126.97156 37.561...",41697.0,정보없음,12.0
1,8930e1d8127ffff,"POLYGON ((126.97254 37.56289, 126.97156 37.561...",36045.0,정보없음,0.0


In [53]:
## 필요 column 추출
build_a = build_grid[['h3_9', 'geometry']]

In [54]:
## 그리드별 가장 높은 건물만 추출
a = build_grid.groupby('h3_9', as_index = False)[['HEIGHT']].max()

In [55]:
## 전체 데이터 + 높은 건물 데이터 결합
b = pd.merge(build_a, a, how = 'left', on = 'h3_9')

In [56]:
b = b.drop_duplicates(['h3_9'], keep = 'first')
b.reset_index(inplace = True) # index reset
b.drop('index', axis = 1, inplace = True) # drop in|dex

In [57]:
build = gpd.GeoDataFrame(b)
build = build.set_crs(epsg = 4326)

In [58]:
build['HEIGHT'] = build['HEIGHT'].fillna(0.0)

In [60]:
build.drop('geometry', axis = 1, inplace = True)
build = pd.DataFrame(build)

### 토지

In [12]:
## 서울시 토지 이용 현황 데이터 불러오기
path = 'data/토지데이터/NGII_LUM_11_서울/'
file_list = os.listdir(path)
file_list = [file for file in file_list if file.endswith('.shp')]

land = gpd.GeoDataFrame()

for i in file_list:
    data = gpd.read_file(path + i, crs = '5181')
    land = pd.concat([land,data])
    
land = land.reset_index(drop = True)

In [16]:
## 토지 정보 데이터 불러오기
land_info = pd.read_excel('data/토지이용현황도_분류항목(코드).xls', header = 5)

In [17]:
## 대분류 정리
land_info['대분류'][:4] = "농지"; land_info['대분류'][4:13] = "임지"
land_info['대분류'][13:32] = "도시 및 주거지"; land_info['대분류'][32:] = "수계"

C:\Users\user\AppData\Local\Temp\ipykernel_22388\2534223690.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  land_info['대분류'][:4] = "농지"; land_info['대분류'][4:13] = "임지"
C:\Users\user\AppData\Local\Temp\ipykernel_22388\2534223690.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  land_info['대분류'][13:32] = "도시 및 주거지"; land_info['대분류'][32:] = "수계"


In [18]:
## 필요 column 추출
land_info = land_info[['대분류', '코드']]

In [19]:
land_info = land_info.astype({'코드' : 'str'}) # merge 위해 자료형 맞춰주기

In [20]:
## 토지 이용 현황 + 토지 정보 데이터 결합
land = pd.merge(land, land_info, how = "inner", left_on = "UCB", right_on = "코드")

In [21]:
## GeoDataFrame 형식으로 변환
land = gpd.GeoDataFrame(land, geometry = 'geometry', crs = '5181')
land = land.to_crs(4326)

In [23]:
## 서울시 H3 그리드의 중심점 추출
seoul_cen = gpd.GeoDataFrame(seoul_grid, geometry = "geometry", crs = "4326")
seoul_cen['geometry'] = seoul_cen['geometry'].centroid

C:\Users\user\AppData\Local\Temp\ipykernel_22388\3796017722.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  seoul_cen['geometry'] = seoul_cen['geometry'].centroid


In [24]:
## 토지 데이터 + 그리드 중심점 데이터 결합
land_grid = gpd.sjoin(seoul_cen, land, how = "left")

In [25]:
## 필요 column 추출
land = land_grid[['h3_9', '대분류', '코드', 'geometry']]

In [63]:
land.drop('geometry', axis = 1, inplace = True)
land = pd.DataFrame(land)

C:\Users\user\AppData\Local\Temp\ipykernel_22388\2373297437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  land.drop('geometry', axis = 1, inplace = True)


### 토지 + 건물

In [76]:
total = pd.merge(build, land, on = 'h3_9')

In [78]:
total

,h3_9,HEIGHT,대분류,코드
0,8930e0a5423ffff,0.00,도시 및 주거지,3210
1,8930e1d8127ffff,89.60,도시 및 주거지,3430
2,8930e1c24afffff,15.15,임지,2210
3,8930e1c3447ffff,22.47,도시 및 주거지,3430
4,8930e1d9d1bffff,0.00,도시 및 주거지,3430
...,...,...,...,...
37575,8930e15a3dbffff,0.00,NaN,NaN
37576,8930e15a237ffff,0.00,NaN,NaN
37577,8930e15a3d3ffff,0.00,NaN,NaN
37578,8930e15a223ffff,0.00,NaN,NaN


In [82]:
total['코드'] = total['코드'].fillna('0000')
total['대분류'] = total['대분류'].fillna('알수없음')

In [84]:
## h3 index를 polygon으로 변환
def to_polygon(l) :
    return Polygon(h3.h3_to_geo_boundary(l, geo_json = True))

total['geometry'] = total['h3_9'].apply(to_polygon)
# total = total.set_crs(epsg = 4326)

In [86]:
df = gpd.GeoDataFrame(total, geometry = 'geometry', crs = '4326')

In [87]:
def polygon_to_coordinates(x) :
    lon, lat = x.exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

In [88]:
df['coordinates'] = df['geometry'].apply(polygon_to_coordinates)
del df['geometry']

In [89]:
df = pd.DataFrame(df)

In [90]:
df['color'] = df['코드'].str[:1]

In [91]:
d = df[df['color'] == "0"].index
df.drop(d, inplace = True)

In [92]:
df.loc[df["color"] == "1", "color_R"] = 0
df.loc[df["color"] == "1", "color_G"] = 255
df.loc[df["color"] == "1", "color_B"] = 0

df.loc[df["color"] == "2", "color_R"] = 255
df.loc[df["color"] == "2", "color_G"] = 255
df.loc[df["color"] == "2", "color_B"] = 0

df.loc[df["color"] == "3", "color_R"] = 255
df.loc[df["color"] == "3", "color_G"] = 0
df.loc[df["color"] == "3", "color_B"] = 255

df.loc[df["color"] == "4", "color_R"] = 0
df.loc[df["color"] == "4", "color_G"] = 0
df.loc[df["color"] == "4", "color_B"] = 255

In [93]:
import pydeck as pdk

In [94]:
layer = pdk.Layer(
    "PolygonLayer",
    df,
    get_polygon = "coordinates",
    get_fill_color = '[1 * color_R,  1 * color_G , 1 * color_B]',
    pickable = True,
    auto_highlight = True
)

# 서울시청 주소
lat = 37.5662; lng = 126.9787
seoul = [lat, lng]

view_state = pdk.ViewState(
    longitude = seoul[1],
    latitude = seoul[0],
    zoom = 14
)

r = pdk.Deck(layers = [layer], initial_view_state = view_state)
r.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{\n  "initialViewState": {…